In [26]:
#!/usr/bin/env python3


import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import patsy

from datetime import timedelta
import logging
import numpy as np
import pandas as pd
from time import time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [27]:
##Replace with your path to the Proj2_Data folder on GoogleColab
path_to_data='C:/Users/zhaij005/Desktop/uiuc/cs598-Practical Statistical Learning/project 2/Proj2_Data'
DATA_DIR ='C:/Users/zhaij005/Desktop/uiuc/cs598-Practical Statistical Learning/project 2/Proj2_Data'

In [28]:
# Configure logging
logging.basicConfig(
    filename=f'{DATA_DIR}/colab_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%H:%M:%S'
)

In [29]:
# Suppress warnings
warnings.filterwarnings('ignore')

In [30]:
# Feature Engineering
def preprocess(data):
    """Preprocess the data by handling missing values and creating time-based features."""
    data.fillna(0, inplace=True)
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Yr2'] = data.Yr ** 2
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])
    data['IsHoliday'] = data['IsHoliday'].apply(int)
    return data

In [31]:
i = 4

In [32]:
        df = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/train.csv')
        test = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/test.csv')
        
                # Convert dates to datetime
        df['Date'] = pd.to_datetime(df['Date'])
        test['Date'] = pd.to_datetime(test['Date'])

        # Extract week, year, and other useful features
        df['Wk'] = df['Date'].dt.isocalendar().week
        #train['Yr'] = train['Date'].dt.year
        df['Wk'] = df['Date'].dt.isocalendar().week
        df['Yr'] = df['Date'].dt.year

In [33]:
        conditions = [
            ((df['Yr'] == 2010) & (df['Wk'] == 51)),
            ((df['Yr'] == 2010) & (df['Wk'] == 52)),
            ((df['Yr'] == 2011) & (df['Wk'] == 51)),
            ((df['Yr'] == 2011) & (df['Wk'] == 52))
        ]
        choices = [5, 2, 3, 4]
        df['Is_Christmas'] = np.select(conditions, choices, default=0)

In [34]:
df[df['Is_Christmas'] > 0] 

,Store,Dept,Date,Weekly_Sales,IsHoliday,Wk,Yr,Is_Christmas
46,1,1,2010-12-24,55931.23,False,51,2010,5
47,1,1,2010-12-31,19124.58,True,52,2010,2
137,1,2,2010-12-24,59889.32,False,51,2010,5
138,1,2,2010-12-31,35819.83,True,52,2010,2
228,1,3,2010-12-24,12080.36,False,51,2010,5
...,...,...,...,...,...,...,...,...
266966,45,95,2010-12-31,43149.88,True,52,2010,2
267056,45,97,2010-12-24,6767.35,False,51,2010,5
267057,45,97,2010-12-31,5881.22,True,52,2010,2
267139,45,98,2010-12-24,222.48,False,51,2010,5


In [35]:
# Function to mark holiday weeks
def add_holiday_flags(df):
    df['Is_SuperBowl'] = (df['Wk'] == 6).astype(int)
    df['Is_Thanksgiving'] = (df['Wk'] == 47).astype(int)
    
    ########################################################################################################
    #### try to assign weight of sales based on understanding of how christmas season works ################
    conditions = [
            ((df['Yr'] == 2010) & (df['Wk'] == 51)),
            ((df['Yr'] == 2010) & (df['Wk'] == 52)),
            ((df['Yr'] == 2011) & (df['Wk'] == 51)),
            ((df['Yr'] == 2011) & (df['Wk'] == 52))
        ]
    choices = [5, 2, 3, 3]
    df['Is_Christmas'] = np.select(conditions, choices, default=0)
    ########################################################################################################
    df['Is_Pre_ChristmasEve'] = (
         ((df['Date'] >= pd.to_datetime('2010-12-23')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2010-12-23')))
        |((df['Date'] >= pd.to_datetime('2011-12-23')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2011-12-23')))
        |((df['Date'] >= pd.to_datetime('2012-12-23')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2012-12-23')))
                                ).astype(int)
    df['Is_After_Christmas'] = (
         ((df['Date'] >= pd.to_datetime('2010-12-26')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2010-12-26')))
        |((df['Date'] >= pd.to_datetime('2011-12-26')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2011-12-26')))
        |((df['Date'] >= pd.to_datetime('2012-12-26')) & (df['Date'] - pd.Timedelta(days=7) < pd.to_datetime('2012-12-26')))
                                ).astype(int)
    
    return df

In [42]:
def train_svd(num_folds_start,num_folds):
    """Train the model using SVD for smoothing."""
    start_time = time()
    #num_folds = 10
    
    for i in range(num_folds_start,num_folds):
        fold_start = time()
        logging.info(f"Processing fold {i+1}/{num_folds}")
        test_pred = pd.DataFrame()

        # Read data for current fold
        train = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/train.csv')
        test = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/test.csv')
        
                # Convert dates to datetime
        train['Date'] = pd.to_datetime(train['Date'])
        test['Date'] = pd.to_datetime(test['Date'])

        # Extract week, year, and other useful features
        train['Wk'] = train['Date'].dt.isocalendar().week
        train['Yr'] = train['Date'].dt.year
        test['Wk'] = test['Date'].dt.isocalendar().week
        test['Yr'] = test['Date'].dt.year
        
        train = add_holiday_flags(train)
        test = add_holiday_flags(test)
        
        # Sort by Store, Dept, and Date for lag feature creation
        train = train.sort_values(['Store', 'Dept', 'Date'])

        # Create lagged features
        train['Lag_1'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
        train['Lag_2'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2)
        train['Rolling_Mean_4'] = train.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(4).mean())
        train.dropna(inplace=True)  # Drop rows with NaN lagged features
        
            
        departments = train['Dept'].unique()
        logging.info(f"Processing {len(departments)} departments for fold {i+1}")
        
        # Process each department
        for dept_idx, department in enumerate(departments, 1):
            # if dept_idx % 5 == 0:  # Log every 5th department
            #     logging.info(f"  Progress: {dept_idx}/{len(departments)} departments")
                
            filtered_train = train[train['Dept'] == department]
            selected_columns = filtered_train[['Store', 'Date', 'Weekly_Sales']]

            # Create pivot table and perform SVD
            X_pivot = selected_columns.pivot(index='Date', columns='Store', 
                                          values='Weekly_Sales').fillna(0)        
            X_matrix = X_pivot.values
            date_mean = X_matrix.mean(axis=1, keepdims=True)
            X_centered = X_matrix - date_mean
            X_centered

            # SVD computation
            U, D, Vt = np.linalg.svd(X_centered, full_matrices=False)
            n_comp = 8
            D_tilda = np.zeros_like(D)
            D_tilda[:n_comp] = D[:n_comp]
            X_tilda = U[:, :n_comp] @ np.diag(D_tilda[:n_comp]) @ Vt[:n_comp, :]
            X_smoothed = X_tilda + date_mean
          

            # Convert back to DataFrame
            X_smoothed_df = pd.DataFrame(X_smoothed, index=X_pivot.index,
                                       columns=X_pivot.columns).reset_index()
            
            # X_original_format = X_smoothed_df.melt(id_vars=['Store'], 
            #                                      var_name='Date', 
            #                                      value_name='Weekly_Sales')

            X_original_format = X_smoothed_df.melt(
                                                 id_vars=['Date'],
                                                 var_name='Store',
                                                 value_name='Weekly_Sales')
            X_original_format['Date'] = pd.to_datetime(X_original_format['Date'])
            X_original_format['Store'] = X_original_format['Store'].astype('int64')
            
            null_values = X_original_format.isnull().sum()
            
            X_original_format['Dept'] = department
            X_original_format = X_original_format.sort_values(
                by=['Store', 'Date']).reset_index(drop=True)
            X_original_format = X_original_format[['Store','Date','Weekly_Sales','Dept']]
            
            

            # Prepare train-test pairs
            train_pairs = X_original_format[['Store', 'Dept']].drop_duplicates(
                ignore_index=True)
            test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
            unique_pairs = pd.merge(train_pairs, test_pairs, 
                                  how='inner', on=['Store', 'Dept'])
            
            # Process training data
            train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
            train_split = preprocess(train_split)
            
            # Create model matrices
            X = patsy.dmatrix('Weekly_Sales + Store + Dept  + IsHoliday+ Yr + Wk + Is_Christmas',
                              data = train_split,
                              return_type='dataframe')
            
            
            
            train_split = dict(tuple(X.groupby(['Store', 'Dept'])))

            # Process test data
            test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
            test_split = preprocess(test_split)
            
            
            
            X = patsy.dmatrix('Store + Dept + IsHoliday +Yr + Wk + Is_Christmas',
                                data = test_split,
                                return_type='dataframe')
            
            X['Date'] = test_split['Date']
            test_split = dict(tuple(X.groupby(['Store', 'Dept'])))
            keys = list(train_split)

         

            # Train and predict for each store-department combination
            for key in keys:
                X_train = train_split[key]
                
                # blend sample size to match expectation of the target formula - did not work
                # row_to_repeat =  X_train[X_train['IsHoliday'] == 1]
                # repeated_row_df = pd.concat([row_to_repeat] * 4, ignore_index=True)
                # X_train = pd.concat([X_train, repeated_row_df], ignore_index=True)

                
                X_test = test_split[key]

                #X_train['Intercept'] = 1.0
                #print(X_train.columns)
                #X_test['Intercept'] = 1.0

                X_test_copy = X_test.copy()
                X_train_copy = X_train.copy()

                corr_mtx = X_train.corr()[['Weekly_Sales']]

                ## variable selection seem to only improve the performance of the first week (no real week and year info to use)
                if i+1 ==1:
                    cor_threshold = 0.16
                else:
                    cor_threshold = 0
                    
                cols_to_keep = ['Intercept'] + corr_mtx[abs(corr_mtx['Weekly_Sales'])>cor_threshold].index.tolist()
                #print(len(cols_to_keep))
                if len(cols_to_keep) ==1:
                    #bascially no model for cases where no good predictor identified
                    cols_to_keep = X_train.columns.tolist()
                #print(f'for key = {key}; high var columns- {cols_to_keep}')

                
                Y = X_train_copy['Weekly_Sales'] #.clip( upper=X_train['Weekly_Sales'].quantile(wzn_perc))
                #Y = X_train['Weekly_Sales'].clip( upper=wzn_perc)
                X_train = X_train[[col for col in cols_to_keep if col not in ['Weekly_Sales','Store', 'Dept','IsHoliday']]]
                X_test = X_test[[col for col in cols_to_keep if col not in ['Weekly_Sales','Store', 'Dept','IsHoliday']]]
 

                #X_train = X_train.drop(['Weekly_Sales','Store', 'Dept','IsHoliday'], axis=1)

                cols_to_drop = X_train.columns[(X_train == 0).all()]
                X_train = X_train.drop(columns=cols_to_drop)
                X_test = X_test.drop(columns=cols_to_drop)
                cols_to_drop = []
                for j in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
                    col_name = X_train.columns[j]
                    # Extract the current column and all previous columns
                    tmp_Y = X_train.iloc[:, j].values
                    tmp_X = X_train.iloc[:, :j].values

                    coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
                    if np.sum(residuals) < 1e-16:
                            cols_to_drop.append(col_name)
                print(cols_to_drop)
                X_train = X_train.drop(columns=cols_to_drop)
                X_test = X_test.drop(columns=cols_to_drop)

                wzn_perc = 0.999
                Y = Y.clip(upper=Y.quantile(wzn_perc))
                model = sm.OLS(Y, X_train).fit()
                mycoef = model.params.fillna(0)

                # Make predictions
                tmp_pred = X_test_copy[['Store', 'Dept', 'Date', 'IsHoliday']]
                X_test = X_test.drop(['Store', 'Dept', 'Date', 'IsHoliday'], axis=1, errors='ignore')

                tmp_pred['Weekly_Pred'] =  np.dot(X_test, mycoef)
                # if i+1 == 5:
                #     tmp_pred['Weekly_Pred'] = np.where((  ((tmp_pred['Store'] == 35) | (tmp_pred['Store']== 10)) &
                #                                          ((tmp_pred['Date']== pd.to_datetime('2011-11-25'))) & 
                #                                          (tmp_pred['Dept']==72)), (640000), tmp_pred['Weekly_Pred'])


                
                test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

            test_pred['Weekly_Pred'].fillna(0, inplace=True)

            test_pred['Wk'] = test_pred['Date'].dt.isocalendar().week
            test_pred['Yr'] = test_pred['Date'].dt.year

            test_pred = add_holiday_flags(test_pred)

            
            test_pred.to_csv(f'{DATA_DIR}/fold_{i+1}/mypred.csv', index=False)
            

                # Fit Gradient Boosting model 
                # model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
                # model.fit(X_train, Y)
                
        fold_time = time() - fold_start
        logging.info(f"Completed fold {i+1} in {fold_time:.1f} seconds")
        print(f"Completed fold {i+1} in {fold_time:.1f} seconds")
    
    total_time = time() - start_time
    logging.info(f"Completed all folds in {total_time:.1f} seconds")




In [43]:

def evaluate(num_folds_start,num_folds):
    """Evaluate the model's predictions."""
    logging.info("Starting evaluation...")
    test_with_label = pd.read_csv(f'{DATA_DIR}/test_with_label.csv')
    #num_folds = 10
    wae = []

    for i in range(num_folds_start,num_folds):
        # Read test data and predictions
        test = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/test.csv')
        test = test.merge(test_with_label, on=['Date', 'Store', 'Dept'])
        test_pred = pd.read_csv(f'{DATA_DIR}/fold_{i+1}/mypred.csv')
        
        # Merge and calculate weighted absolute error
        new_test = test.merge(test_pred, on=['Date', 'Store', 'Dept'], how='left')
        actuals = new_test['Weekly_Sales'].fillna(0)
        preds = new_test['Weekly_Pred'].fillna(0)
        weights = new_test['IsHoliday_x'].apply(lambda x: 5 if x else 1)

        new_test['weights'] = weights 
        new_test['errors'] = actuals - preds
        new_test['errors_abs'] = abs(actuals - preds)
        new_test['errors_weighted'] = weights * abs(actuals - preds)
        new_test.to_excel(f'{DATA_DIR}/fold_{i+1}/myvalidate.xlsx', index=False)
        
        wae_score = sum(weights * abs(actuals - preds)) / sum(weights)
        wae.append(wae_score)
        print(f"Fold {i+1} WAE: {wae_score:.3f}")
        logging.info(f"Fold {i+1} WAE: {wae_score:.3f}")

    avg_wae = sum(wae) / len(wae)
    logging.info(f"Average WAE across all folds: {avg_wae:.3f}")
    print(f"Average WAE across all folds: {avg_wae:.3f}")
    
    return wae


In [44]:
def main():
    """Main function to run the Walmart price prediction model."""
    start_time = time()
    logging.info("Starting Walmart Price Prediction")
    
    logging.info("Training models with SVD smoothing...")
    train_svd(4,5)
    
    logging.info("Evaluating predictions...")
    wae = evaluate(4,5)
    
    total_time = time() - start_time
    logging.info(f"Processing complete! Total time: {total_time:.1f} seconds")

if __name__ == "__main__":
    main()

['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas']
['Is_Christmas